<a href="https://colab.research.google.com/github/Mallikarjuna9/GenAI_Hackathon/blob/main/GenAI_Hackathon_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/Mallikarjuna9/GenAI_Hackathon.git

Cloning into 'GenAI_Hackathon'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 287.33 KiB | 3.68 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 2.8 MB/s eta 0:00:00


In [48]:
from getpass import getpass

key = getpass('Please enter your together AI API Key here: ')

Please enter your together AI API Key here: ··········


setting the key as environment variable

In [49]:
import os

os.environ['TOGETHER_API_KEY'] = key

setting the client to Together

In [5]:
from together import Together

client = Together()

Checking the basic functionality of the client LLM

In [50]:
chat_response = client.chat.completions.create(
    messages=[{'role':"user", 'content':"What is the AI in 3 lines?"}],
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    max_tokens=1000,
)

print(chat_response.choices[0].message.content)

Artificial Intelligence (AI) refers to computer systems that can perform tasks that typically require human intelligence. AI systems use algorithms and data to learn, reason, and make decisions, often autonomously. Examples of AI include virtual assistants, image recognition, and natural language processing.


Chaniging the working directory

In [14]:
os.chdir(r"/content/GenAI_Hackathon")

In [15]:
import pandas as pd

df = pd.read_csv(r"train.csv")

In [16]:
df

,ticket_ID,ticket_subject,ticket_body,department,type,priority,language
0,1001,Discrepancia de facturación en Google Workspace,"Estimado equipo de soporte de TI,\n\nEstoy esc...",Billing and Payments,Incident,low,es
1,1002,Urgent Consultation Request for Critical IT Is...,"Dear IT Services Support Team, I hope this mes...",Customer Service,Request,high,en
2,1003,Consulta sobre Servicios de Consultoría en TI,"Estimado Servicio de Atención al Cliente,\n\nM...",General Inquiry,Request,medium,es
3,1004,Demande de mise à jour des dossiers,"Cher service client, \n\nJe vous écris pour de...",Human Resources,Change,low,fr
4,1005,Issues with Slack connection affecting team co...,"Dear Customer Support Team,\n\nI am encounteri...",Product Support,Problem,medium,en
5,1006,Defective Dell XPS 13 9310,"Dear Tech Online Store Support,\n\nI received ...",Returns and Exchanges,Incident,low,en
6,1007,Touchscreen and Keyboard Issues with Surface P...,"Dear Tech Online Store Customer Support,\n\nI ...",Sales and Pre-Sales,Problem,medium,en
7,1008,AWS-Serverausfall,"Sehr geehrte Kundenbetreuung,\n\nwir hatten ei...",Service Outages and Maintenance,Incident,high,de
8,1009,Solicitud urgente de orientación y solución de...,"Estimado soporte al cliente, espero que este m...",Technical Support,Request,high,es


Defining the functions for prompt_template and Classify ticket


In [55]:
model = "meta-llama/Meta-Llama-3-70B-Instruct-Lite"     #setting the model name

## Function to classify the ticket which takes message prompt and model as the input and returns the response
def ClassifyTicket(messages,model=model):
    response = client.chat.completions.create(messages = messages,model=model,max_tokens=1000)
    return response.choices[0].message.content

#Returns the prompt which is suitable for the model
def prompt_template(ticket_subject, ticket_body,system_prompt):
    message = [{'role':'system', 'content':system_prompt},{'role':"user", 'content': f'''ticket_subject:{ticket_subject}\n ticket_body:{ticket_body}'''}]
    return message


Defining the few shot examples which model will use to better classification of tickets

In [19]:
dept_train_map = []
type_train_map = []
priority_train_map = []
rows_to_iterate= [1,2,3,6,7]  # setting the rows to iterate which has the unique labels
for idx, row in df.iterrows():
    if idx in rows_to_iterate:
        temp_map = f'''ticket_subject : {row['ticket_subject']}\t\t ticket_body : {row['ticket_body']}\t\t ---> {row['priority']}'''
        priority_train_map.append(temp_map)
        temp_map = f'''ticket_subject : {row['ticket_subject']}\t\t ticket_body : {row['ticket_body']}\t\t ---> {row['department']}'''
        dept_train_map.append(temp_map)
        temp_map = f'''ticket_subject : {row['ticket_subject']}\t\t ticket_body : {row['ticket_body']}\t\t ---> {row['type']}'''
        type_train_map.append(temp_map)

print(dept_train_map)

print(priority_train_map)

print(type_train_map)

['ticket_subject : Urgent Consultation Request for Critical IT Issues\t\t ticket_body : Dear IT Services Support Team, I hope this message finds you well. My name is <name> and I am currently experiencing critical issues with our server administration, which are significantly impacting our operations. We rely heavily on your IT Consulting Service for our ongoing technical needs. Therefore, we urgently need your expert assistance to resolve these issues as quickly as possible. Time is of the essence because our operations have come to a standstill. Please contact us at your earliest convenience, either via email or phone at <tel_num>. The swift resolution of this situation is imperative to minimize any further disruption to our business. Thank you for your prompt attention to this matter. Kind regards, <name>\t\t ---> Customer Service', 'ticket_subject : Consulta sobre Servicios de Consultoría en TI\t\t ticket_body : Estimado Servicio de Atención al Cliente,\n\nMi nombre es <name>, y es

Defining the system prompts which will be used for system role
We're providing the mapping of the ticket subject and body to the department, type and priority to train the model for this task


In [20]:
dept_and_lang_prompt = f"""you need to find the language code and department from the below 10 departments for the user provided ticket subject and body : .
                              - Technical Support
                              - Customer Service
                              - Billing and Payments
                              - Product Support
                              - IT Support
                              - Returns and Exchanges
                              - Sales and Pre-Sales
                              - Human Resources
                              - Service Outages and Maintenance
                              - General Inquiry
                           Use following examples to determine department instead of using your intelligence to find department
                           {dept_train_map[0]},
                           {dept_train_map[1]},
                           {dept_train_map[2]},
                           {dept_train_map[3]},
                           {dept_train_map[4]}.
                           Provide output as list only example output: [department, language code]  and no explanation or analysis"""


prio_prompt = f"""Classify priority of following email content given by user into one of the following priority levels .
                          - low
                          - medium
                          - high
                          Use following examples.
                           {priority_train_map[0]},
                           {priority_train_map[1]},
                           {priority_train_map[2]},
                           {priority_train_map[3]},
                           {priority_train_map[4]}.
                           Return the priority level only and no explanation or analysis"""


type_prompt = f"""Classify the following email content given by user into one of these types.
                          - Incident
                          - Request
                          - Problem
                          - Change
                          Use following examples.
                           {type_train_map[0]},
                           {type_train_map[1]},
                           {type_train_map[2]},
                           {type_train_map[3]},
                           {type_train_map[4]}.
                           Return the type only and no explanation or analysis"""

In [21]:
#intializing the variables for checking basic function calling
idx=8
ticket_id = df.ticket_ID[idx]
ticket_subject = df.ticket_subject[idx]
ticket_body = df.ticket_body[idx]

In [22]:
#checking the prompt_template function which returns the input prompt for the classify ticket function
message = prompt_template(ticket_subject, ticket_body,dept_and_lang_prompt)
message

[{'role': 'system',
  'content': "you need to find the language code and department from the below 10 departments for the user provided ticket subject and body : .\n                              - Technical Support\n                              - Customer Service\n                              - Billing and Payments\n                              - Product Support\n                              - IT Support\n                              - Returns and Exchanges\n                              - Sales and Pre-Sales\n                              - Human Resources\n                              - Service Outages and Maintenance\n                              - General Inquiry\n                           Use following examples to determine department instead of using your intelligence to find department\n                           ticket_subject : Urgent Consultation Request for Critical IT Issues\t\t ticket_body : Dear IT Services Support Team, I hope this message finds you well. My name

In [42]:
# Predicting the department and language code of the ticket for a df[8]
resp = ClassifyTicket(prompt_template(ticket_subject,ticket_body,dept_and_lang_prompt))
resp

'[Technical Support, es]'

In [43]:
# Predicting the priority of the ticket for a df[8]
resp = ClassifyTicket(prompt_template(ticket_subject,ticket_body,prio_prompt))
resp

'high'

In [44]:
# Predicting the type of the ticket for a df[8]
resp = ClassifyTicket(prompt_template(ticket_subject,ticket_body,type_prompt))
resp

'Request'

A function to compare the output of the model and the actual values in the dataset and increments the accuracy variables values if matched. Returns the variables

In [32]:
def evaluate_acc(resp, tick_id, dept_acc,type_acc,prio_acc,lang_acc, count,df):
    dept_acc = dept_acc + int(resp[0]==df[df['ticket_ID'] == int(tick_id)]['department'].values[0])
    type_acc = type_acc + int(resp[1]==df[df['ticket_ID'] == int(tick_id)]['type'].values[0])
    prio_acc = prio_acc + int(resp[2]==df[df['ticket_ID'] == int(tick_id)]['priority'].values[0])
    lang_acc = lang_acc + int(resp[3]==df[df['ticket_ID'] == int(tick_id)]['language'].values[0])
    count+=1
    return dept_acc, type_acc, prio_acc, lang_acc, count

Defining the final_score function to calculate the accuracy of each labels and returns the final score based on the recommended criteria

In [33]:
def Final_score(dept_acc, type_acc, prio_acc, lang_acc,count):  # count indicates number of tickets
    dept_acc=dept_acc/count
    type_acc=type_acc/count
    prio_acc=prio_acc/count
    lang_acc=lang_acc/count
    print("Department Accuracy: ", dept_acc)
    print("Type Accuracy: ", type_acc)
    print("Priority Accuracy: ", prio_acc)
    print("Language Accuracy: ", lang_acc)
    return (0.4*dept_acc + 0.3*type_acc + 0.2*prio_acc + 0.1*lang_acc)

Running the model on validation data and printing accuracy

In [56]:
#intializing accuracy variables to find accuracy
dept_acc = 0
lang_acc = 0
prio_acc = 0
type_acc = 0
count = 0

rows_to_iter_val=[0,4,5,8]    # row indexes to validate which are not fed to the model while training
for idx,row in df.iterrows():          # iterating over rows which are needed to validate
    if idx in rows_to_iter_val:
        tick_id = row['ticket_ID']
        respDL=ClassifyTicket(prompt_template(row['ticket_subject'],row['ticket_body'],dept_and_lang_prompt))  # Returns the string of department and language in a list
        respT=ClassifyTicket(prompt_template(row['ticket_subject'],row['ticket_body'],type_prompt))   # return the type of ticket
        respP=ClassifyTicket(prompt_template(row['ticket_subject'],row['ticket_body'],prio_prompt))   # return the priority of ticket
        respDL=respDL.strip("[]").strip().split(", ")      # since returned value is a string, changing it to a list by splitting the string
        resp=[respDL[0],respT,respP,respDL[1]]
        dept_acc, type_acc, prio_acc, lang_acc, count = evaluate_acc(resp, tick_id,dept_acc, type_acc, prio_acc, lang_acc, count,df)     # checks the predicted values with the actual values and returns the count and accuracy variables
        print('predicted output  '+str(tick_id)+' : '+str(resp))
        print('Actual output     '+str(tick_id)+' : '+str([str(df[df['ticket_ID'] == int(tick_id)]['department'].values[0]), str(df[df['ticket_ID'] == int(tick_id)]['type'].values[0]), str(df[df['ticket_ID'] == int(tick_id)]['priority'].values[0]), str(df[df['ticket_ID'] == int(tick_id)]['language'].values[0])]))

print('Final score : '+str(Final_score(dept_acc,type_acc,prio_acc,lang_acc,count)))   # prints the final score of the output

predicted output  1001 : ['Billing and Payments', 'Incident', 'medium', 'es']
Actual output     1001 : ['Billing and Payments', 'Incident', 'low', 'es']
predicted output  1005 : ['Technical Support', 'Request', 'medium', 'en']
Actual output     1005 : ['Product Support', 'Problem', 'medium', 'en']
predicted output  1006 : ['Returns and Exchanges', 'Incident', 'high', 'en']
Actual output     1006 : ['Returns and Exchanges', 'Incident', 'low', 'en']
predicted output  1009 : ['IT Support', 'Request', 'high', 'es']
Actual output     1009 : ['Technical Support', 'Request', 'high', 'es']
Department Accuracy:  0.5
Type Accuracy:  0.75
Priority Accuracy:  0.5
Language Accuracy:  1.0
Final score : 0.625


Reading test.csv file and storing in a test_df dataframe

In [35]:
test_df = pd.read_csv(r'test.csv')
test_df.head()

,ticket_ID,ticket_subject,ticket_body
0,1,Urgente: problema con la funzione di condivisi...,"Gentile assistenza clienti,\n\nSto riscontrand..."
1,2,Abrechnungsdiskrepanzen in Google Workspace,"Liebes IT-Support-Team,\n\nMit diesem Schreibe..."
2,3,Problèmes avec la fonction sans fil de l'Epson...,"Bonjour à l'équipe d'assistance,\n\nJ'ai un pr..."
3,4,NaN,"Gentile assistenza clienti, \n\nSpero che ques..."
4,5,Assistenza necessaria per gli arresti anomali ...,Gentile assistenza clienti di Tech Online Stor...


Creating a Dummy result data frame to store
Creating a function to write the predicted labels to the result data frame

In [46]:
# Creating a result_df data frame with columns ticket_ID, department, type, priority, language to store predicted labels
result_df = pd.DataFrame(columns=['ticket_ID', 'department', 'type', 'priority', 'language'])

# Function to add multiple tickets to the DataFrame
def add_tickets(df, tickets):
    # Create a DataFrame from the list of tickets
    new_tickets_df = pd.DataFrame(tickets, columns=['ticket_ID', 'department', 'type', 'priority', 'language'])
    # Append the new tickets to the existing DataFrame
    df = pd.concat([df, new_tickets_df], ignore_index=True)
    return df

result_df.head()

,ticket_ID,department,type,priority,language


Iterating over rows in the test_df, and calling model to predict the labels with the appropriate prompt template and adding the predicted labels to the result_df data frame

In [47]:
for idx, row in test_df.iterrows():       # iterating over each row of test_df to predict labels
    tick_id = str(row['ticket_ID'])
    respDL=ClassifyTicket(prompt_template(row['ticket_subject'],row['ticket_body'],dept_and_lang_prompt)) # to return department and language
    respT=ClassifyTicket(prompt_template(row['ticket_subject'],row['ticket_body'],type_prompt))  # to return type
    respP=ClassifyTicket(prompt_template(row['ticket_subject'],row['ticket_body'],prio_prompt)) # to return priority
    respDL=respDL.strip("[]").strip().split(", ")   # splitting string to get department and language in a list
    resp=[tick_id,respDL[0],respT,respP,respDL[1]]
    print(resp)
    result_df = add_tickets(result_df, [resp])    # add predicted labels to data frame

['1', 'Technical Support', 'Incident', 'high', 'it']
['2', 'Billing and Payments', 'Problem', 'medium', 'de']
['3', 'Technical Support', 'Problem', 'medium', 'fr']
['4', 'Billing and Payments', 'Request', 'medium', 'it']
['5', 'Technical Support', 'Problem', 'medium', 'it']
['6', 'Technical Support', 'Problem', 'medium', 'it']
['7', 'Customer Service', 'Request', 'high', 'it']
['8', 'Technical Support', 'Problem', 'high', 'pt']
['9', 'IT Support', 'Request', 'medium', 'es']
['10', 'Technical Support', 'Problem', 'medium', 'it']
['11', 'Product Support', 'Problem', 'medium', 'it']
['12', 'Technical Support', 'Problem', 'medium', 'it']
['13', 'Billing and Payments', 'Problem', 'medium', 'it']
['14', 'Technical Support', 'Change', 'high', 'de']
['15', 'IT Support', 'Incident', 'high', 'de']
['16', 'IT Support', 'Problem', 'high', 'it']
['17', 'Technical Support', 'Request', 'medium', 'pt']
['18', 'Technical Support', 'Problem', 'medium', 'de']
['19', 'Service Outages and Maintenance', 'Re

KeyboardInterrupt: 

In [39]:
result_df.head(-10)

,ticket_ID,department,type,priority,language
0,1,Technical Support,Incident,high,it
1,2,Billing and Payments,Problem,medium,de
2,3,Technical Support,Problem,medium,fr
3,4,Billing and Payments,Request,medium,it
4,5,Technical Support,Problem,medium,it
...,...,...,...,...,...
273,274,Product Support,Problem,medium,pt
274,275,Technical Support,Incident,high,fr
275,276,Technical Support,Problem,high,de
276,277,Technical Support,Problem,medium,hi


Saving result data frame to csv file

In [ ]:
result_df.to_csv(f'tickets3.csv', index=False)